### 提升requests模块的爬取效率

- 多线程和多进程(不建议使用)
- 线程池和进程池(适当使用)
- 单线程+异步协程(推荐使用)

In [9]:
import re
import random
import requests
from lxml import etree
from multiprocessing.dummy import Pool

url = "https://www.pearvideo.com/category_31"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.75 Safari/537.36"
}

def download_video(url):
    # 向视频URL发送请求，获取二进制数据
    return requests.get(url=url, headers=headers).content

def save_video(data):
    # 将视频二进制数据保存到本地
    video_name = './' + str(random.randint(1000, 9999)) + ".mp4"
    with open(video_name, 'wb') as f:
        f.write(data)

page_text = requests.get(url=url, headers=headers).text

tree = etree.HTML(page_text)

li_list = tree.xpath('//ul[@id="listvideoListUl"]/li')

# 实例化线程池对象
pool = Pool(4)

video_url_list = list()
for li in li_list:
    detail_url = "https://www.pearvideo.com/" + li.xpath('./div/a/@href')[0]
    page_text = requests.get(url=detail_url, headers=headers).text
    ex = 'srcUrl="(.*?)",vdoUrl='
    
    video_url = re.findall(ex, page_text, re.S)[0]
    video_url_list.append(video_url)

# 下载视频的二进制数据，存放到一个列表中video_data_list
video_data_list = pool.map(download_video, video_url_list)

# 将视频的二进制数据保存到本地(真正地下载视频到本地)
pool.map(save_video, video_data_list)

[None, None, None, None]

### 协程回顾

event_loop：事件循环，相当于一个无限循环，我们可以把一些函数注册到这个事件循环上，当满足某些条件的时候，函数就会被循环执行。程序是按照设定的顺序从头执行到尾，运行的次数也是完全按照设定。当在编写异步程序时，必然其中有部分程序的运行耗时是比较久的，需要先让出当前程序的控制权，让其在背后运行，让另一部分的程序先运行起来。当背后运行的程序完成后，也需要及时通知主程序已经完成任务可以进行下一步操作，但这个过程所需的时间是不确定的，
需要主程序不断的监听状态，一旦收到了任务完成的消息，就开始进行下一步。loop就是这个持续不断的监视器。

coroutine：中文翻译叫协程，在 Python 中常指代为协程对象类型，我们可以将协程对象注册到事件循环中，它会被事件循环调用。我们可以使用 async 关键字来定义一个方法，这个方法在调用时不会立即被执行，而是返回一个协程对象。

task：任务，它是对协程对象的进一步封装，包含了任务的各个状态。

future：代表将来执行或还没有执行的任务，实际上和 task 没有本质区别。

In [12]:
# 基本使用
import asyncio

# 定义了一个特殊函数
async def hello(name):
    print("hello " + name)

# 通过函数名加括号执行这个特殊函数，并不会立即执行，而是返回一个协程对象
c = hello("oldboy")

# 实例化一个事件循环对象
loop = asyncio.get_event_loop()

# 将协程对象注册到事件循环中, 并执行事件循环
loop.run_until_complete(c)

RuntimeError: This event loop is already running

In [ ]:
# task的使用
import asyncio

# 定义了一个特殊函数
async def hello(name):
    print("hello " + url)

# 通过函数名加括号执行这个特殊函数，并不会立即执行，而是返回一个协程对象
c = hello("oldboy")

# 实例化一个事件循环对象
loop = asyncio.get_event_loop()

# 创建一个task任务对象
task = loop.create_task(c)

# 将协程对象注册到事件循环中, 并执行事件循环
print(task)
loop.run_until_complete(task)
print(task)

In [ ]:
# feture的使用
import asyncio

# 定义了一个特殊函数
async def hello(name):
    print("hello " + url)

# 通过函数名加括号执行这个特殊函数，并不会立即执行，而是返回一个协程对象
c = hello("oldboy")

# 实例化一个事件循环对象
loop = asyncio.get_event_loop()

# 创建一个feture任务对象
feture = asyncio.ensure_future(c)

# 将协程对象注册到事件循环中, 并执行事件循环
print(feture)
loop.run_until_complete(feture)
print(feture)

In [ ]:
# 回调函数的使用
import asyncio

# 定义了一个特殊函数
async def request(url):
    print("向URL发送请求，获取响应数据" + url)
    return "page_text"

# 定义一个回调函数，用来实现数据解析
def call_back(task):
    print("在这里进行数据解析操作")
    print(task.result())


# 通过函数名加括号执行这个特殊函数，并不会立即执行，而是返回一个协程对象
c = request("https://www.baidu.com")

# 实例化一个事件循环对象
loop = asyncio.get_event_loop()

# 创建一个feture任务对象
feture = asyncio.ensure_future(c)

# 加上一个绑定回调
feture.add_done_callback(call_back)

# 将协程对象注册到事件循环中, 并执行事件循环
loop.run_until_complete(feture)